 Les actions RDD sont des opérations qui déclenchent l'exécution du traitement des données décrit par les transformations RDD et renvoient les résultats au programme pilote Spark. Ce processus est souvent appelé un travail. Contrairement aux transformations qui définissent les opérations à effectuer, les actions collectent ou génèrent les résultats des calculs RDD. Ci-dessous, nous examinerons différents types d'actions fréquemment utilisées dans les applications Spark.

In [1]:
from pyspark import SparkConf, SparkContext

In [2]:
# conf
conf = SparkConf().setMaster("local").setAppName("Action Notebook")
sc = SparkContext.getOrCreate(conf = conf)

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.UnsupportedOperationException: getSubject is supported only if a security manager is allowed
	at java.base/javax.security.auth.Subject.getSubject(Subject.java:347)
	at org.apache.hadoop.security.UserGroupInformation.getCurrentUser(UserGroupInformation.java:588)
	at org.apache.spark.util.Utils$.$anonfun$getCurrentUserName$1(Utils.scala:2446)
	at scala.Option.getOrElse(Option.scala:201)
	at org.apache.spark.util.Utils$.getCurrentUserName(Utils.scala:2446)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:339)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:59)
	at java.base/jdk.internal.reflect.DirectConstructorHandleAccessor.newInstance(DirectConstructorHandleAccessor.java:62)
	at java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:501)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.ja

In [ ]:
numbersRDD = sc.parallelize((1, 2, 3, 4, 5))

In [ ]:
# collect() est l'une des actions les plus courantes, qui récupère l'intégralité des données RDD vers le programme pilote. Puisqu’il récupère toutes les données, il doit être utilisé avec prudence, en particulier avec de grands ensembles de données.
collectedNumbers = numbersRDD.collect()
print(", ".join(map(str, collectedNumbers)))

In [ ]:
# count() renvoie le nombre d'éléments dans le RDD.
totalCount = numbersRDD.count()
print(totalCount)

In [ ]:
# take(n) renvoie les n premiers éléments du RDD.
firstThree = numbersRDD.take(3)
print(", ".join(map(str, firstThree)))

In [ ]:
# first() est similaire à take(1)mais elle est plus lisible et renvoie le premier élément du RDD.
firstElement = numbersRDD.first()
print(firstElement)

In [ ]:
# reduce() effectue une opération binaire associative et commutative spécifiée sur les éléments RDD et renvoie le résultat.
sum = numbersRDD.reduce(lambda a,b : a+b)
print(sum)

In [ ]:
# fold() est similaire à reduce(), mais prend une « valeur zéro » supplémentaire à utiliser pour l'appel initial sur chaque partition. La « valeur zéro » doit être l'élément d'identité de l'opération.
foldedResult = numbersRDD.fold(0, lambda a, b: a + b)
print(foldedResult)

In [ ]:
# MAGIC aggregate() est encore plus générale que fold()et reduce(). Il faut une « valeur zéro » initiale, une fonction pour combiner les éléments du RDD avec l'accumulateur, et une autre fonction pour fusionner deux accumulateurs.

# COMMAND ----------

aggregateResult = numbersRDD.aggregate(
    (0, 0),  # zeroValue
    lambda acc, value: (acc[0] + value, acc[1] + 1),  # seqOp
    lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1])  # combOp
)
print(aggregateResult)

In [ ]:
# Mise en cache et persistance
# Lorsque des actions RDD sont appelées, Spark calcule le RDD et sa lignée. Si un RDD est utilisé plusieurs fois, il est efficace de le conserver en mémoire en appelant la méthode persist()ou cache(). De cette façon, Spark peut réutiliser le RDD pour des actions ultérieures sans recalculer l'intégralité du lignage.
numbersRDD.cache() # Persist the RDD in memory
sumCached = numbersRDD.reduce(lambda a, b: a + b) # Uses the cached data
print(sumCached)